## Import Required Libraries
Running cell below will import the required libraries being used in the rest of the cells of this notebook

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm 
import sys

In [ ]:
# exp_no = 'E1'
exp_no = 'E2'
# exp_no = 'E3'

In [ ]:
from dataclasses import dataclass

@dataclass
class Parameters:
    dummy                   : bool  = False
    exp_no                  : str   = exp_no         
    model_type              : str   = ''
    pretrained_model_type   : str   = ''
    background_type         : str   = ''
params = Parameters()

def get_params_from_exp(params):
    if params.exp_no=='E1':
        params.model_type              = 'real'
        params.pretrained_model_type   = 'resnet'      
        params.background_type         = 'gray'
    elif params.exp_no=='E2':
        params.model_type              = 'ideal'
        params.pretrained_model_type   = 'resnet'      
        params.background_type         = 'gray'
        params.file_name               = 'exp_E2'
    elif params.exp_no=='E3':
        params.model_type              = 'real'
        params.pretrained_model_type   = 'swin_trans_vit'      
        params.background_type         = 'gray'
    return params

params = get_params_from_exp(params)
params.file_name = f'exp_{params.exp_no}'

def get_exp_header():
    print('='*95)
    print('| ',f'{"MODEL_TYPE":15}','|',f'{"PRETRAINED_MODEL":22}','|',f'{"BACKGROUND_TYPE":22}','|',f'{"FILENAME":22}','|')
    print('-'*95)
    print('| ',f'{params.model_type:15}','|',f'{params.pretrained_model_type:22}','|',f'{params.background_type:22}','|',f'{params.file_name:22}','|')

get_exp_header()


### PARAMS & Paths
Below cell contains the parameters switching between the prediction model, background replacement strategy, and other few parameters

In [ ]:

# use_subset = False
use_subset = True

if use_subset:
    subset_count = 100

params = get_params_from_exp(params)
get_exp_header()

In [ ]:
submission_version    =  False

global path_seg_maps_raw,path_img_val,DS_name
path_codes    = os.getcwd()
path_notebook = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

DS_name_annotation    = 'ImageNetS'
DS_name_img           = 'ImageNet'

# path_ds_main   = f'{path_notebook}/dataset'   if submission_version else 'D:\DS\ImageNet'
path_ds_main   = 'D:\DS\ImageNet'

if not submission_version:
    path_repos      = 'E:/PHD/datacloud_data/repos' 
    codes_local     = os.getcwd()
    
if submission_version:
    path_repos     = os.path.join(os.getcwd(),'utils')
    codes_local    = path_codes
sys.path.append(path_repos)
submission_version    =  True

path_utils    = os.path.abspath(os.path.join(path_notebook,'utils'))

path_seg_maps  = os.path.join(path_ds_main,f'{DS_name_annotation}/ILSVRC2012_img_val_S/ImageNetS50/validation-segmentation')
path_img_val   = os.path.join(path_ds_main,f'{DS_name_img}/ILSVRC2012_img_val')

filenames,files = [],[]
tagnames      = os.listdir(path_seg_maps)
for tn in tagnames:
    sub_dir = f'{path_seg_maps}//{tn}'
    filenames.extend(os.listdir(sub_dir))
    tempp = []
    for fn in os.listdir(sub_dir):
        tempp.append(f'{sub_dir}//{fn}')
    files.extend(tempp)

suffix_full = f'{params.pretrained_model_type}_{params.model_type}_{params.background_type}'

suffix                     = f'{params.model_type}_{params.background_type}'

results_main                   = os.path.join(codes_local,'results')
# results_ = 'E:\\PHD\\datacloud_data\\codes\\shap_bpt\\results_review'
codes_path     = os.getcwd()


results_path               = os.path.join(results_main,params.pretrained_model_type)
path_html                  = os.path.join(results_path,suffix_full)
results_path               = os.path.join(results_path,suffix_full)
plots_path                 = os.path.join(results_path,'plots')
plotsIoU_path              = os.path.join(results_path,'plots_IoU')

# path_csv          = os.path.join(codes_path,     'csv')
path_csv    = '../results_logits_with_auc_clipped/'

path_notebook = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
path_utils    = os.path.abspath(os.path.join(path_notebook,'utils'))

print(results_path)
print(path_html)

get_exp_header()

In [ ]:
path_csv,plots_path,results_path,results_main

## LOAD CSV FILE GENERATED BY `notebooks/N2_Run_experiments.ipynb`

In [ ]:
csv_filename = f'{path_csv}/csv_{params.file_name}_{DS_name_img}_{params.pretrained_model_type}_{suffix}.csv'
csv_filename,os.path.exists(csv_filename)

In [ ]:
df = pd.read_csv(csv_filename)
df.head(3)

In [ ]:
#     name,                 color,                  functor
methods = [ 
    'BPT-100',
    'BPT-500',   
    'BPT-1000',  
    ]
methods_pe = [ # if single background
    'Partition-100', 
    'Partition-500',  
    'Partition-1000',  
    ]
methods_aa = [ # if multiple backgrounds
    'AA-100',   
    'AA-500',
    'AA-1000', 
]
methods_aa_huge = [
    'AA-5000', 
    'AA-10000',
]
methods_lime = [
    'LIME-50',
    'LIME-100',
    'LIME-200',  
    ]
methods_cam = [
    'GradCAM',  
    'aIDG',       
    'aGradExpl', 
    ]
methods += methods_pe if params.background_type=='gray' and params.model_type!='ideal' else methods_aa
if params.model_type == 'ideal':
    methods += methods_aa_huge

methods += methods_lime
if params.model_type != 'ideal':
    methods += methods_cam

In [ ]:
methods

In [ ]:
auc_methods      = ['aucI_pred', 'aucD_pred', 'aucI_mse', 'aucD_mse', 'max_IoU', 'au_IoU', 'time_exp']

## GENERATE/LOAD CSS AND JS FILE

In [ ]:
GENERATE_FILE = True
if GENERATE_FILE:
    with open(f'{path_utils}/html_stylesheet.txt', 'w') as f:
        f.write("""
                <style>
                    table.space {margin-top: 10px;padding-top: 10px;width: 100%;} 
                    table{border: 1px solid black;},
                    th{font-size: 2em},tr{font-size: 2em},
                    .collapsible-title {
                      font-size: 18px;
                      font-weight: bold;
                      color: #333;
                    }
                    .collapsible-text {
                      font-size: 16px;
                      color: #666;
                      line-height: 1.5;
                    }
                    .collapsible-content {
                      font-size: 20px;}
                    table {
                      text-align: left;
                      position: relative;
                      border-collapse: separate; 
                    border-top: 2px solid;
                    border-bottom: 2px solid;
                    border-right: 2px solid;
                    }
                    td {
                    padding: 0.1rem;
                    }
                    th {
                        border: 2px solid black;
                        padding: 0.1rem;
                    }
                    tr.red th {
                      background: red;
                      color: white;
                    }
                    tr.green th {
                      background: green;
                      color: white;
                    }
                    tr.purple th {
                      background: purple;
                      color: white;
                    }
                    th {
                    border-radius: 3px;
                    background-color: rgba(255, 255, 255, 0.8) !important;
                    text-align:center;
                    background: white;
                    position: sticky;
                    top: 0;
                    box-shadow: 0 2px 2px -1px rgba(0, 0, 0, 0.4);
                    }
                    tr:hover {background-color: rgba(234, 242, 215,0.8);}
                    .switch {
                    position: relative;
                    display: inline-block;
                    width: 60px;
                    height: 34px;
                    }
                    .switch input {
                    opacity: 0;
                    width: 0;
                    height: 0;
                    }
                    .slider {
                    position: absolute;
                    cursor: pointer;
                    top: 0;
                    left: 0;
                    right: 0;
                    bottom: 0;
                    background-color: #808080;
                    -webkit-transition: .4s;
                    transition: .4s;
                    }
                    .slider:before {
                    position: absolute;
                    content: "";
                    height: 26px;
                    width: 26px;
                    left: 4px;
                    bottom: 4px;
                    background-color: white;
                    -webkit-transition: .4s;
                    transition: .4s;
                    }
                    input:checked + .slider {
                    background-color: #2196F3;
                    }
                    input:focus + .slider {
                    box-shadow: 0 0 1px #2196F3;
                    }
                    input:checked + .slider:before {
                    -webkit-transform: translateX(26px);
                    -ms-transform: translateX(26px);
                    transform: translateX(26px);
                    }
                    .slider.round {
                    border-radius: 34px;
                    }
                    .slider.round:before {
                    border-radius: 50%;
                    }
                    #topBtn {
                        position: fixed;
                        bottom: 5px;
                        float: right;
                        right: 18.5%;
                        left: 0.25%;
                        max-width: 70px;
                        width: 100%;
                        font-size: 18px;
                        border-color: rgba(85, 85, 85, 0.8);
                        background-color: rgba(27, 163, 156,0.9);
                        padding: 2px;
                        border-radius: 10px;
                        }
                    #topBtn:hover {
                        background-color: #7dbbf1;
                    }
                    table {border: none;}
                    * {
                    box-sizing: border-box;
                    }
                    .box {
                    float: left;
                    width: 100%;
                    padding: 10px;
                    }
                    .clearfix::after {
                    content: "";
                    clear: both;
                    display: table;
                    }
                    table {
                    height: 100%;
                    border-collapse: seperate;
                    width: 100%;
                    margin: 10px;
                    font-size: 0.8em;
                    }
                    thead {
                    border-radius: 3px;
                    position: sticky;
                    top: 0;
                    background: #eee;
                    }
                    header h1 {
                        font-size: 70px;
                        font-weight: 600;
                        background-image: linear-gradient(to left, #553c9a, #b393d3);
                        color: transparent;
                        background-clip: text;
                        -webkit-background-clip: text;
                        text-shadow: 0px 4px 10px rgba(179, 147, 211, 0.8)
                    }
                    h1 {
                      font-size: 30px;
                      line-height: 1.8;
                      text-transform: uppercase;
                      font-family: "Montserrat", sans-serif;
                    }
                .hero {
                    position: relative;
                    background: #333 url(http://srdjanpajdic.com/slike/2.jpg) no-repeat center center fixed;
                    -webkit-background-size: cover;
                    -moz-background-size: cover;
                    background-size: cover;
                    text-align: center;
                    color: #fff;
                    padding-top: 20px;
                    min-height: 300px;
                    letter-spacing: 2px;
                    font-family: "Montserrat", sans-serif;
                    h1 {
                    font-size: 50px;
                    line-height: 1.3;
                    span {
                    font-size: 25px;
                    color: $color2;
                    border-bottom: 2px solid $color2;
                    padding-bottom: 8px;
                    line-height: 3;
                    }
                }
                .main_container{
                   width:99%;
                }
                html, body {
                height:100%;
                width:99%;
                margin:0;
                padding:0;
                }
                    </style>
                    """)
    with open(f'{path_utils}/html_head.txt', 'w') as f:
        f.write("""
            <head>
                <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css"
                    rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" 
                    crossorigin="anonymous">
                <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/js/bootstrap.bundle.min.js"
                    integrity="sha384-MrcW6ZMFYlzcLA8Nl+NtUVF0sA7MsXsP1UyJoMp4YLEuNSfAP+JcXn/tWtIaxVXM" crossorigin="anonymous">
                </script>
            </head>
            """)
    with open(f'{path_utils}/html_footer_scripts_full.txt', 'w') as f:
        f.write("""
            <script>const toggleAllButton = document.getElementById('toggleAllButton');
                const collapsibleContents = document.querySelectorAll('.collapsible-content');
                toggleAllButton.addEventListener('click', () => {
                  collapsibleContents.forEach(content => {
                    if (content.style.display === 'none') {
                      content.style.display = 'block';
                      content.previousElementSibling.querySelector('.toggle-icon').textContent = '';
                    } else {
                      content.style.display = 'none';
                      content.previousElementSibling.querySelector('.toggle-icon').textContent = '';
                    }
                  });
                });
            </script>
            """)
    with open(f'{path_utils}/html_footer_scripts_IoU.txt', 'w') as f:
        f.write("""<script>
        const toggleAllButton = document.getElementById('toggleAllButton');
            const collapsibleContents = document.querySelectorAll('.collapsible-content');
            toggleAllButton.addEventListener('click', () => {
              collapsibleContents.forEach(content => {
              document.getElementById('data_rows').style.border = 'none';
                if (content.style.display === 'none') {
                  content.style.display = 'block';
                  content.previousElementSibling.querySelector('.toggle-icon').textContent = '';
                } else {
                  content.style.display = 'none';
                  content.previousElementSibling.querySelector('.toggle-icon').textContent = '';
                }
              });
            });
            </script>
           """)

In [ ]:
with open(f"{path_utils}/html_stylesheet.txt", "r+") as f:
    html_stylesheet = f.read()
with open(f"{path_utils}/html_footer_scripts_IoU.txt", "r+") as f:
    html_footer_scripts_IoU = f.read()
with open(f"{path_utils}/html_footer_scripts_full.txt", "r+") as f:
    html_footer_scripts_full = f.read()
with open(f"{path_utils}/html_head.txt", "r+") as f:
    html_head = f.read()

In [ ]:
get_exp_header()

## HTML FULL

In [ ]:
sel_thresh  = 1.01
###################################################
if exp_no=='E1' or exp_no=='E2':
    suff_exp = f'{params.background_type}'
else:
    suff_exp = f'{params.background_type}_heatmap'
#########################################################################################################
if use_subset:
    files_set = files[:subset_count]
else:
    files_set = files
###################################################

html_f_name = f'{path_html}/HTML_{params.exp_no}_{params.model_type}_{params.pretrained_model_type}_{params.background_type}_{len(files_set)}.html'
print(params.exp_no)
print(html_f_name)
print(path_html)
print(plots_path,os.path.exists(plots_path))

###################################################
version = 'full' if sel_thresh>1 else 'low'
total_images = len(np.unique(df[df.f_S<sel_thresh].image))
nf_images = []
print(f'Total Images: {total_images}\t Total Methods {len(methods)}')
if os.path.exists(html_f_name):
    os.remove(html_f_name)
with open(html_f_name, 'a') as f:
    f.write('<html>')
    f.write(html_head)
    f.write('''
        <body>
        <div class='main_container'>
            <div class="hero">
                <h1><span>Webpage containing all IoU results [Visual(heatmaps) and Numerical] for </span>
                    <br>Shapley image explanations<br>
                    with data-aware Binary Partition Trees.
                </h1>
                ''')
    f.write(f'<h3>Params: [model: {params.pretrained_model_type}, background_type: {params.background_type}] - No of Images:{len(files_set)}</h3>')
    f.write('''
                <div class="mouse">
                    <span></span>
                </div>
            </div>
            ''')
    f.write(html_stylesheet)
with open(html_f_name, 'a') as f:
        f.write('''
                <fieldset>
                    <legend>Toggle Explanation Details:</legend>
                    <label class="switch">
                          <input type="checkbox" id="toggleAllButton" checked>
                          <span class="slider round"></span>
                    </label>
                </fieldset>
                <br><br>
                <table id="table-1"><tr>
                    <th align= "center"> Input Image</th> 
                    <th align= "center"> GrTruth</th> 
                ''')
for meth in methods:
    if meth in ['AA-100','AA-500']:
        # print(meth)
        meth = f'&nbsp{meth}&nbsp'
    meth= '|GrExpl|' if meth=='GradExpla' else meth
    meth= '  GrCAM  ' if meth=='GradCAM' else meth
    with open(html_f_name, 'a') as f:
        f.write(f'<th align= "center"> {meth}</th>')

for fff_id,fff in enumerate(tqdm(files_set)):
    file_n = fff.split('.')[0].split('//')[-1]
    if file_n not in df.image.values:
        nf_images.append(file_n)
        continue
    # img_path = f'plots//{file_n}_{background_type}_heatmap.png'
    # img_path = f'{plots_path}//{file_n}_{background_type}_heatmap.png'
    img_path = f'plots/{file_n}_{suff_exp}.png'
    if not os.path.exists(img_path):
        pass;
    PP = df[df.image==file_n].f_S.iloc[0]
    # if version=='low':
    #     if PP>0.5:
    #         continue
    PL = df[df.image==file_n].pred_lbl.iloc[0]
    with open(html_f_name, 'a') as f:
        f.write(f'<tr><td colspan="{len(methods)+2}">\
                <img src={img_path} width="99.9%"> <br>\
                    <p  style="font-size:22">{fff_id+1} - {file_n}  -  {PL}  -  {PP:0.5}  -  \
                    </p>\
                </td>\
                ')
    with open(html_f_name, 'a') as f:
            f.write(f'\
                <tr><td">\
                  ')
    for sm in auc_methods:
        df1 = df[(df.image==file_n)].eval(f'{sm}')
        with open(html_f_name, 'a') as f:
            f.write(f'\
                    <tr><td>\
                    <div class="collapsible-container">\
                      <div class="collapsible-header">\
                        <span class="toggle-icon"></span>\
                      </div>\
                      <div class="collapsible-content">\
                        <p>\
                        <b>{sm}</b>\
                        </div>\
                    </div>\
                    </td>\
                    ')
        for meth_id,meth in enumerate(methods):
            df2 = df1.iloc[meth_id]
            with open(html_f_name, 'a') as f:
                f.write(f'\
                        <td> \
                        <div class="collapsible-container">\
                          <div class="collapsible-header">\
                            <span class="toggle-icon"></span>\
                          </div>\
                          <div class="collapsible-content">\
                            <p>\
                            {df2:0.5} </p>\
                            </div>\
                        </div>\
                        </td>\
                        ')
        with open(html_f_name, 'a') as f:
            f.write(f'\
            </tr>\
            ')
    with open(html_f_name, 'a') as f:
        f.write('''
                    </div>
                </div>
                </td></tr>
                ''')
with open(html_f_name, 'a') as f:
    f.write('''
            </tr>
            </table>
            <button id="topBtn"><a href="#top" style="color: white">Top</a></button>
            ''')
    f.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>')
    f.write(html_footer_scripts_full)
    f.write('''  
                </body>
            </html>
            ''')
print(f'Data Written in File :\t{html_f_name}')

print(f'Total images:\t\t{len(files):<25}\nMatched images:\t\t{total_images}\nUnmatched images\t{len(nf_images)}')
print(f'Data Written in File :\t{html_f_name}')

## IoU HTML FILE

In [ ]:
if exp_no=='E1' or exp_no=='E2':
    suff_exp = f'{params.background_type}'
else:
    suff_exp = f'{params.background_type}_IoU'
#########################################################################################################
if use_subset:
    files_set = files[:subset_count]
else:
    files_set = files
#########################################################################################################
html_f_name = f'{path_html}/HTML_{exp_no}_{params.model_type}_{params.pretrained_model_type}_{params.background_type}_IoU_{len(files_set)}.html'
print(exp_no)
print(html_f_name)
print(path_html)
print(plots_path,os.path.exists(plots_path))
#########################################################################################################
sel_thresh  = 1.01
version = 'full' if sel_thresh>1 else 'low'
total_images = len(np.unique(df[df.f_S<sel_thresh].image))
auc_methods_sel = auc_methods[-3:-1]

print(f'Total Images: {total_images}\t Total Methods {len(methods)}')

if os.path.exists(html_f_name):
    os.remove(html_f_name)
with open(html_f_name, 'a') as f:
    f.write('\
    <html>\
    ')
    f.write(html_head)
    f.write('''
        <body>
        <div class='main_container'>
            <div class="hero">
                <h1><span>Webpage containing all IoU results [Visual(IoU) and Numerical] for </span>
                    <br>Shapley image explanations<br>
                    with data-aware Binary Partition Trees.
                </h1>
                ''')
    f.write(f'<h3>Params: [model: {params.pretrained_model_type}, background_type: {params.background_type}] - No of Images:{len(files_set)}</h3>')
    f.write('''
                <div class="mouse">
                    <span></span>
                </div>
            </div>
            ''')
    f.write(html_stylesheet)
# (0.0, 0.0, 0.75) # True Positives
# (1.0, 0.6, 0.2) # False Positives
# (1.0, 0.4, 1.0) # False Negatives
with open(html_f_name, 'a') as f:
        f.write('''
        <fieldset>
            <legend>Toggle Explanation Details:</legend>
            <label class="switch">
                  <input type="checkbox" id="toggleAllButton" checked>
                  <span class="slider round"></span>
            </label>
        </fieldset>
        <br>
        ''')
        f.write(f'\
        <table id="table-1"><tr>\
            <thead>\
            <tr>\
            <div class="clearfix">\
                <th colspan="{len(methods)+2}">\
                    <div class="box">\
                        LEGEND\
                    </div>\
                </th>\
            </tr>\
            <tr>\
                <th colspan=4>\
                    <div class="box" style="background-color:#0000bf; color: #ffffff;">\
                        True Positive\
                    </div>\
                </th>\
                <th colspan=5>\
                    <div class="box" style="background-color:#ff9933">\
                        False Negative\
                    </div>\
                </th>\
                <th colspan=5>\
                    <div class="box" style="background-color:#ff66ff">\
                        False Positive\
                    </div>\
                </th>\
                </th>\
            </div>\
        </tr>\
        <tr>\
        <th colspan="{len(methods)+2}">\
            <div class="box">\
                Methods\
            </div>\
        </th>\
        </tr>\
            <tr>\
            <th align= "center"> Input Image</th> \
            <th align= "center"> GrTruth</th> \
                ')
for meth in methods:
    if meth in ['AA-100','AA-500']:
        meth = f'&nbsp{meth}&nbsp'
    meth= '|GrExpl|' if meth=='GradExpla' else meth
    meth= '  GrCAM  ' if meth=='GradCAM' else meth
    with open(html_f_name, 'a') as f:
        f.write(f'<th align= "center"> {meth}</th>\
        ')

with open(html_f_name, 'a') as f:
    f.write('\
    </thead>\
    ')
     
for fff_id,fff in enumerate(tqdm(files_set)):
    file_n = fff.split('.')[0].split('//')[-1]
    if file_n not in df.image.values:
        # print(f'image not found - {file_n}')
        continue

    # img_path = f'plots_IoU//{file_n}_{background_type}.png'
    # img_path = f'{plotsIoU_path}//{file_n}_{background_type}_IoU.png'
    img_path = f'plots_IoU/{file_n}_{suff_exp}.png'
    PP = df[df.image==file_n].f_S.iloc[0]
    # if version=='low':
    #     if PP>0.5:
    #         continue
    PL = df[df.image==file_n].pred_lbl.iloc[0]
    
    with open(html_f_name, 'a') as f:
        f.write(f'<tr><td colspan="{len(methods)+2}">\
                <img src={img_path} width="99.9%"> <br>\
                    <p style="font-size:22">{fff_id+1} - {file_n}  -  {PL}  -  {PP:0.5}  -  \
                    </p>\
                </td>\
                ')
    with open(html_f_name, 'a') as f:
            f.write(f'\
                <tr><td">\
                  ')
    for sm in auc_methods_sel:
        df1 = df[(df.image==file_n)].eval(f'{sm}')
        with open(html_f_name, 'a') as f:
            f.write(f'\
                    <tr><td>\
                    <div class="collapsible-container">\
                      <div class="collapsible-header">\
                        <span class="toggle-icon"></span>\
                      </div>\
                      <div class="collapsible-content">\
                        <p>\
                        <b>{sm}</b>\
                        </div>\
                    </div>\
                    </td>\
                    ')
        for meth_id,meth in enumerate(methods):
            df2 = df1.iloc[meth_id]
            with open(html_f_name, 'a') as f:
                f.write(f'\
                        <td id="data_rows"> \
                        <div class="collapsible-container">\
                          <div class="collapsible-header">\
                            <span class="toggle-icon"></span>\
                          </div>\
                          <div class="collapsible-content">\
                            <p>\
                            {df2:0.5} </p>\
                            </div>\
                        </div>\
                        </td>\
                        ')
        with open(html_f_name, 'a') as f:
            f.write(f'\
            </tr>\
            ')
    with open(html_f_name, 'a') as f:
        f.write('\
                    </div>\
                </div>\
                </td></tr>\
                ')
with open(html_f_name, 'a') as f:
    f.write('''
                </tr>
            </table>
            <button id="topBtn"><a href="#top" style="color: white">Top</a></button>
            ''')
    f.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>')
    f.write(html_footer_scripts_IoU)
    f.write('''
                    </div>
                </body>
            </html>
            ''')
print(f'Data Written in File :\tfile://{os.path.abspath(html_f_name)}')

print(f'Total images:\t\t{len(files):<25}\nMatched images:\t\t{total_images}\nUnmatched images\t{len(nf_images)}')
print(f'Data Written in File :\t{html_f_name}')
print('img_path: ',img_path)

# Both

In [ ]:
sel_thresh  = 1.01
###################################################
if exp_no=='E1' or exp_no=='E2':
    suff_exp = f'{params.background_type}'
    suff_exp_heatmap = f'{params.background_type}'
    suff_exp_iou = f'{params.background_type}'
else:
    suff_exp_heatmap = f'{params.background_type}_heatmap'
    suff_exp_iou = f'{params.background_type}_IoU'
#########################################################################################################
if use_subset:
    files_set = files[:subset_count]
else:
    files_set = files
###################################################

html_f_name = f'{path_html}/HTML_{exp_no}_{params.model_type}_{params.pretrained_model_type}_{params.background_type}_combined_{len(files_set)}.html'
print(params.exp_no)
print(html_f_name)
print(path_html)
print(plots_path,os.path.exists(plots_path))

###################################################
version = 'full' if sel_thresh>1 else 'low'
total_images = len(np.unique(df[df.f_S<sel_thresh].image))
nf_images = []
print(f'Total Images: {total_images}\t Total Methods {len(methods)}')
if os.path.exists(html_f_name):
    os.remove(html_f_name)
with open(html_f_name, 'a') as f:
    f.write('<html>')
    f.write(html_head)
    f.write('''
        <body>
        <div class='main_container'>
            <div class="hero">
                <h1><span>Webpage containing all IoU results [Visual(heatmaps) and Numerical] for </span>
                    <br>Shapley image explanations<br>
                    with data-aware Binary Partition Trees.
                </h1>
                ''')
    f.write(f'<h3>Params: [model: {params.pretrained_model_type}, model_type: {params.model_type} ,background_type: {params.background_type}] - No of Images:{len(files_set)}</h3>')
    f.write('''
                <div class="mouse">
                    <span></span>
                </div>
            </div>
            ''')
    f.write(html_stylesheet)
with open(html_f_name, 'a') as f:
        f.write('''
                <fieldset>
                    <legend>Toggle Explanation Details:</legend>
                    <label class="switch">
                          <input type="checkbox" id="toggleAllButton" checked>
                          <span class="slider round"></span>
                    </label>
                </fieldset>
                <br><br>
                <table id="table-1"><tr>
                    <th align= "center"> Input Image</th> 
                    <th align= "center"> GrTruth</th> 
                ''')
for meth in methods:
    if meth in ['AA-100','AA-500']:
        # print(meth)
        meth = f'&nbsp{meth}&nbsp'
    meth= '|GrExpl|' if meth=='GradExpla' else meth
    meth= '  GrCAM  ' if meth=='GradCAM' else meth
    with open(html_f_name, 'a') as f:
        f.write(f'<th align= "center"> {meth}</th>')

for fff_id,fff in enumerate(tqdm(files_set)):
    file_n = fff.split('.')[0].split('//')[-1]
    if file_n not in df.image.values:
        nf_images.append(file_n)
        continue
    # img_path = f'plots//{file_n}_{background_type}_heatmap.png'
    # img_path = f'{plots_path}//{file_n}_{background_type}_heatmap.png'
    # img_path = f'plots/{file_n}_{suff_exp}.png'
    img_path = f'plots_red/{file_n}_{suff_exp_heatmap}.png'
    iou_path = f'plots_IoU_red/{file_n}_{suff_exp_iou}.png'

    if not os.path.exists(img_path):
        pass;
    PP = df[df.image==file_n].f_S.iloc[0]
    # if version=='low':
    #     if PP>0.5:
    #         continue
    PL = df[df.image==file_n].pred_lbl.iloc[0]
    with open(html_f_name, 'a') as f:
        f.write(f'<tr><td colspan="{len(methods)+2}">\
                <img src={img_path} width="99.9%"> <br>\
                <img src={iou_path} width="99.9%"> <br>\
                    <p style="font-size:22">{fff_id+1} - {file_n}  -  {PL}  -  {PP:0.5}  -  \
                    </p>\
                </td>\
                ')
    with open(html_f_name, 'a') as f:
            f.write(f'\
                <tr><td">\
                  ')
    for sm in auc_methods:
        df1 = df[(df.image==file_n)].eval(f'{sm}')
        with open(html_f_name, 'a') as f:
            f.write(f'\
                    <tr><td>\
                    <div class="collapsible-container">\
                      <div class="collapsible-header">\
                        <span class="toggle-icon"></span>\
                      </div>\
                      <div class="collapsible-content">\
                        <p>\
                        <b>{sm}</b>\
                        </div>\
                    </div>\
                    </td>\
                    ')
        for meth_id,meth in enumerate(methods):
            df2 = df1.iloc[meth_id]
            with open(html_f_name, 'a') as f:
                f.write(f'\
                        <td> \
                        <div class="collapsible-container">\
                          <div class="collapsible-header">\
                            <span class="toggle-icon"></span>\
                          </div>\
                          <div class="collapsible-content">\
                            <p>\
                            {df2:0.5} </p>\
                            </div>\
                        </div>\
                        </td>\
                        ')
        with open(html_f_name, 'a') as f:
            f.write(f'\
            </tr>\
            ')
    with open(html_f_name, 'a') as f:
        f.write('''
                    </div>
                </div>
                </td></tr>
                ''')
with open(html_f_name, 'a') as f:
    f.write('''
            </tr>
            </table>
            <button id="topBtn"><a href="#top" style="color: white">Top</a></button>
            ''')
    f.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>')
    f.write(html_footer_scripts_full)
    f.write('''  
                </body>
            </html>
            ''')
print(f'Data Written in File :\t{html_f_name}')

print(f'Total images:\t\t{len(files):<25}\nMatched images:\t\t{total_images}\nUnmatched images\t{len(nf_images)}')
print(f'Data Written in File :\t{html_f_name}')

# Reduce File Size for High Quality Images

In [ ]:
import os
from PIL import Image

# Define input and output directories
input_folder = plots_path
output_folder = f"{plots_path}_red"
new_dpi = (72, 72)  # Set new DPI

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Process all images in the folder
for filename in tqdm(os.listdir(input_folder)):
    if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # Open the image
        with Image.open(input_path) as img:
            # Save with new DPI
            new_size = (img.width // 6, img.height // 6)
            img = img.resize(new_size, Image.LANCZOS)

            img.save(output_path, dpi=new_dpi)
            del img
        # print(f"Processed: {filename}")

print("✅ All images processed and saved with reduced DPI!")


In [ ]:
import os
from PIL import Image

# Define input and output directories
input_folder = plotsIoU_path
output_folder = f"{plotsIoU_path}_red"
new_dpi = (72, 72)  # Set new DPI

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Process all images in the folder
for filename in tqdm(os.listdir(input_folder)):
    if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # Open the image
        with Image.open(input_path) as img:
            # Save with new DPI
            new_size = (img.width // 2, img.height // 2)
            img = img.resize(new_size, Image.LANCZOS)

            img.save(output_path, dpi=new_dpi)
            del img
        # print(f"Processed: {filename}")

print("✅ All images processed and saved with reduced DPI!")
